# Interatomic Potential Repository Content Manager

This is a working Notebook providing a single location for adding and modifying content in the potentials.nist.gov database.

Notebook outline:

0. Setup: Python library imports and defining global parameters.

1. Citations: Load, add and modify citation information associated with potential models. Each potential should have at least one citation, with the first being the "primary" one for generating IDs. For unpublished potentials, use the unpublished citation option.

2. Potentials: Load, add and modify potential information.  A "potential" should be a unique parameterization with an associated citation. Note that saving to the database is done in step #4.

3. Implementations: Load, add and modify the implementations of a potential. An "implementation" is a specific version/representation of a potential in a specific format. Each potential can have zero, one or multiple implementations.

4. Save potentials: Review the new/modified potentials and save the content to the database.

5. Actions: Actions are used to list changes to the repository. If a potential is added/modified, a corresponding action should also be added.

6. Requests: A request is listed on the repository whenever someone asks for a model that we do not have.

## 0. Setup

### Library imports

In [1]:
import uuid
import datetime
import shutil
from pathlib import Path
import socket

import potentials

import numpy as np

# Jupyter display libraries
from IPython.core.display import display, HTML

### Global parameters

- __complete_potentials__ is a compiled list of the potentials whose addition/changes were uploaded to the database.  This is used by step #5 for reporting website changes.  This should be reset after creating an Action for meaningful sets of potentials.

- __db__ is the Database object for accessing and uploading records to the database.  remote_name accesses the saved "potentials" settings, which has sign-in info for https://potentials.nist.gov.

- __workspace__ is the workspace to assign to the uploaded records.  If None, then they are only accessible to the submitter until the workspace is changed.  If 'Global Public Workspace', anyone can see the records after uploading.

In [2]:
complete_potentials = []

In [2]:
db = potentials.Database(local=True, remote=True, remote_name='potentials')

In [3]:
#workspace = None
workspace = 'Global Public Workspace'

- - -

## 1. Citations

### 1.1 Build or load citation

#### Option #1: For existing citations and/or new dois

In [5]:
doi = '10.1088/1361-651X/ac095c'

citation = db.fetch_citation(doi, verbose=True)

Citation retrieved from CrossRef


#### Option #2: For new doi-less citations

In [ ]:
author = 'M. I. Belko and V. E. Gusakov and N. N. Dorozhkin'
year = 2010
ID = note = f'{year}--Belko-V-I-Gusakov-V-E-Dorozhkin-N-N'

citation = potentials.load_record('Citation',
    ENTRYTYPE = 'inproceedings',
    title = 'Potential EDIP for germanium: parameterization and molecular dynamics simulation of point defects',
    author = author,
    ID = ID,
    note = note,
    year = year,
    booktitle = 'Proc. IV International Conference, Materials and Structures of Modern Electronics',
    month='Sep',
    address='Minsk, Belarus', 
    pages= '15--18',
    day='23--24'
)

In [16]:
author = 'M. I. Mendelev'
year = 2021
ID = note = f'{year}--Mendelev-M-I--Fe'
#title = 'Efficient "universal" shifted Lennard-Jones model for all KIM API supported species'
title = 'to be published'

citation = potentials.load_record('Citation',
    ENTRYTYPE = 'unpublished',
    title = title,
    author = author,
    ID = ID,
    note = note,
    year = year,
)

#### Option #3: From local directory

In [ ]:
libdir = Path('E:/website/LAMMPS potentials/2nn/biblib')

citations = []
for bibfile in libdir.glob('*.bib'):
    with open(bibfile) as f:
        citations.append(potentials.load_record('Citation', model=f.read()))

In [ ]:
citation = citations[19]
print(citation.year_authors)

### 1.2 Investigate and modify

- Add abstract

- Check and replace garbled latex or other symbols in authors, title and abstract

- Verify correct year and page number(s)

#### 1.2.1 Generate html and list content fields

In [17]:
print(citation.year_authors)
print()
citation.html(render=True)
print()
for key, value in citation.bib.items():
    print(key+':', value)

2021--Mendelev-M-I




ENTRYTYPE: unpublished
title: to be published
author: M. I. Mendelev
ID: 2021--Mendelev-M-I--Fe
note: 2021--Mendelev-M-I--Fe
year: 2021


#### 1.2.2 Modify and update fields

Any valid bibtex fields can be added, but citation rendering relies only on the primary fields. 

In [7]:
citation.bib['abstract'] = ' '.join(['A semi-empirical interatomic potential for the post-transition metal, bismuth, is developed based on the second nearest-neighbor modified embedded-atom method (MEAM). The potential reproduces a range of physical properties, such as the lattice constant, cohesive energy, elastic constants, vacancy formation energy, surface energy, and the melting point of pure bismuth. The calculations are done for the rhombohedral ground state of Bi. The results show good agreement with density functional theory and experimental data. The developed MEAM potential for bismuth is useful for material and mechanical behavior studies of the pure material at different conditions and sets the stage for the development of interatomic potentials for bismuth alloys or other bismuth compounds.'])
#citation.bib['author'] = 'Ryan S. Elliott and Andrew Akerson'
#citation.bib['volume']
#citation.bib['number']
#citation.bib['title'] = 'Applications of computational thermodynamics - the extension from phase equilibrium to phase transformations and other properties'
#citation.bib['year']
#citation.bib['pages']


#### 1.2.3 Review and Save Citation

In [7]:
citation.html(render=True)

In [ ]:
citation.build_model()

In [18]:
db.save_citation(citation, overwrite=True, verbose=True)
db.upload_citation(citation, workspace=workspace, overwrite=True, verbose=True)

Citation record named 2021--mendelev-m-i--fe added to C:\Users\lmh1\Documents\library
Citation record named 2021--mendelev-m-i--fe added to https://potentials.nist.gov/
Citation record named 2021--mendelev-m-i--fe assigned to workspace Global Public Workspace


In [15]:
#db.delete_citation(citation, local=True, remote=True, verbose=True)

Citation record named 2021--mendelev-m-i--bi deleted from C:\Users\lmh1\Documents\library
Citation record named 2021--mendelev-m-i--bi deleted from https://potentials.nist.gov/


In [ ]:
db.download_citations(overwrite=True, verbose=True)

- - -

## 2. Potentials

### 2.1 Build or load potential

#### Option #1: Build new potential using citation(s)

- __elements__ (*list*) The element models used by the potential.
- __citations__ (*list*) The citation(s) to associate with the potential.
- __notes__ (*str, optional*) Notes providing more information on the potential, such as usage notes and how the potential relates to other potentials.
- __key__ (*str, optional*) UUID4 key to assign to the potential.  If None, a new key will be generated.
- __othername__ (*str, optional*) Specifies an alternate description of the interactions besides the list of elements.  Examples include if the model is "universal", designed for a specific compound, or is a coarse-grain representation.
- __fictional__ (*bool, optional*) Flag indicating if the model is "fictional", i.e. purposefully fit to unrealistic properties. 
- __modelname__ (*str, optional*) Used if citation + element info is not enough to create a unique ID, e.g. a publication lists multiple parameterizations of the same element. The value should make it clear which version it is associated with, and correspond to how the authors refer to it when possible.
- __recorddate__ (*datetime.date, optional*) Date to assign to the record.  If not given, will use today's date.

In [19]:
potential = potentials.load_record('Potential',
    elements=['Fe'],
    citations=[citation],
    notes="This potential was developed to simulate the solidification of Fe under the Earth's inner core conditions.",
    #key=None,
    #othername='water',
    #fictional=True,
    #modelname='34',
    #recorddate=datetime.date(2019, 12, 24)
    )

#### Option #2: Select existing potential

In [49]:
potential = db.get_potential(id='2021--Mendelev-M-I--Fe', verbose=True)

Matching record retrieved from local


### 2.2 Investigate and modify

#### 2.2.1 Generate html and list content fields

In [43]:
display(HTML(f'<H4>{potential.id}</H4>'))
potential.html(render=True)

In [12]:
potential.metadata()

{'name': 'potential.2021--Zhou-H-Dickel-D-E-Baskes-M-I-et-al--Bi',
 'key': '0aa1b65e-0641-4845-bab6-ca6b0ed589ce',
 'id': '2021--Zhou-H-Dickel-D-E-Baskes-M-I-et-al--Bi',
 'recorddate': datetime.date(2021, 7, 16),
 'notes': None,
 'fictional': False,
 'elements': ['Bi'],
 'othername': None,
 'modelname': None,
 'citations': [{'name': '10.1088_1361-651x_ac095c',
   'year_authors': '2021--Zhou-H-Dickel-D-E-Baskes-M-I-et-al',
   'journal': 'Modelling and Simulation in Materials Science and Engineering',
   'title': 'A modified embedded-atom method interatomic potential for bismuth',
   'author': 'Henan Zhou and Doyl E Dickel and Michael I Baskes and Sungkwang Mun and Mohsen Asle Zaeem',
   'pages': '065008',
   'number': '6',
   'volume': '29',
   'publisher': 'IOP Publishing',
   'month': 'jul',
   'year': '2021',
   'url': 'https://doi.org/10.1088%2F1361-651x%2Fac095c',
   'doi': '10.1088/1361-651x/ac095c',
   'ENTRYTYPE': 'article',
   'ID': 'Zhou_2021',
   'abstract': 'A semi-empirical

#### 2.2.2 Modify and update fields

Same fields as above

In [ ]:
#potential.key = 
#potential.recorddate = datetime.date()
#potential.elements = []
#potential.othername = 'water'
#potential.fictional = 
#potential.modelname = 

In [15]:
potential.notes = 'These files were provided by Henan Zhou (Mississippi State) on July 13, 2021 and posted with her permission.'

In [23]:
potential.notes = ' '.join(['This is a modified version of 2018--Farkas-D-Caro-A--Fe-Ni-Cr-Co-Cu that adds the ZBL correction at shorter interatomic distances making it suitable for radiation studies.'])

In [7]:
potential.notes += ' Update: This potential is known to have issues with pure zirconium phases. See 2021--Starikov-S-Smirnova-D--Zr-Nb for an updated parameterization.'

In [ ]:
potential.citations[0] = citation

## 3. Implementations

### 3.1 List all current implementation types

List is given to help keep type values (relatively) uniform. Only needs to be called after new types have been added.

In [13]:
pots = db.get_potentials(remote=False)
imptypes = []
for pot in pots:
    for imp in pot.implementations:
        imptypes.append(imp.type)
imptypes = np.unique(imptypes)

print('All existing implementation types:')
for imptypes in imptypes:
    print('   ', imptypes)

All existing implementation types:
    ADP tabulated functions
    Dynamo MEAM
    EAM setfl
    EAM tabulated functions
    Equations
    FORTRAN
    Finnis-Sinclair tables
    GULP
    IMD option EAM
    LAMMPS pair_style adp
    LAMMPS pair_style aenet (custom)
    LAMMPS pair_style agni
    LAMMPS pair_style bop
    LAMMPS pair_style comb3
    LAMMPS pair_style eam
    LAMMPS pair_style eam/alloy
    LAMMPS pair_style eam/cd
    LAMMPS pair_style eam/fs
    LAMMPS pair_style eam/he
    LAMMPS pair_style edip
    LAMMPS pair_style edip/multi
    LAMMPS pair_style eim
    LAMMPS pair_style extep
    LAMMPS pair_style hybrid table linear 1000 eam/alloy
    LAMMPS pair_style hybrid/overlay eam/alloy eam/fs
    LAMMPS pair_style hybrid/overlay zbl eam/alloy
    LAMMPS pair_style hybrid/overlay zbl snap
    LAMMPS pair_style lcbop
    LAMMPS pair_style meam
    LAMMPS pair_style meam (modified)
    LAMMPS pair_style meam/spline
    LAMMPS pair_style polymorphic
    LAMMPS pair_style reax

### 3.2 Build or load implementation

#### 3.2.1 List ids for current potential and implementations

In [21]:
print('suggested imp prefix:', potential.impid_prefix)
print('implementation ids:')
for i, implementation in enumerate(potential.implementations):
    print(f'{i}: {implementation.id} {implementation.type}')

suggested imp prefix: 2021--Mendelev-M-I--Fe
implementation ids:


#### 3.2.2 Build new implementation

- __type__ (*str*) Describes the type of the implementation. Use one of the listed values above if possible.
- __id__ (*str*) Unique human-readable id for the implementation. For new content, should be derived from the associated potential id:
    - Remove all authors except for the first
    - Add a simple format descriptor: LAMMPS, GULP, table, parameters, FORTRAN, etc.
    - Add a version descriptor: ipr1, ipr2, etc
- __notes__ (*str, optional*) Notes on the implementation.  This includes where the file(s) came from, who created them, testing info, how it differs from other versions, etc.
- __key__ (*str, optional*) UUID4 key to assign to the implementation.  If None, a new key will be generated.
- __status__ (*str, optional*) Indicates the status of the implementation. Available values are 
    - "active" (default) indicates a current implementation.
    - "superseded" indicates an implementation that is still consistent with the potential model, but has minor issues that were fixed by a newer implementation.
    - "retracted" indicates an implementation that was identified as being an invalid representation of the potential model.
- __date__ (*datetime.date, optional*) The date that the implementation was submitted or added.  If not given, will use today's date.

In [22]:
potential.add_implementation(
    type = 'LAMMPS pair_style eam/fs',
    id = f'{potential.impid_prefix}--LAMMPS--ipr1',
    notes = ' '.join(['This file was provided by Mikhail Mendelev on July 15, 2021 and posted with his permission.']),
    #key='0cdc699f-e78a-49db-8606-7a33c98c2184',
    #status=None,
    #date=datetime.date(2020, 6, 18), 
)
index = len(potential.implementations) - 1

In [28]:
kimid = 'MO_959249795837'
implementation = potentials.Implementation(
    type='OpenKIM',
    id=kimid,
    notes=' '.join(['Listing found at https://openkim.org.']),
    #notes=' '.join(['Listing found at https://openkim.org.  This KIM potential is based on a parameter file with identical parameter values as 1989--Tersoff-J--Si-Ge--LAMMPS--ipr1.']),
    #notes=' '.join(['Listing found at https://openkim.org. This KIM potential is implemented from the analytical expressions rather than a tabulated parameter file.'])#  The parameter file that this KIM potential is based on has slightly different values due to precision rounding than 2017--Purja-Pun-G-P--Si--LAMMPS--ipr1.']),
    key='da76eb36-7ca1-421a-bd64-8f6bdd905b15',
    #status=None,
    #date=datetime.date(2020, 6, 18), 
)
implementation.add_link(url=f"https://openkim.org/id/{kimid}", linktext = kimid)

index = len(potential.implementations)
potential.implementations.append(implementation)



In [23]:
potential.html(render=True)

#### 3.2.3 Select implementation

Must be done

In [45]:
index = 0
implementation = potential.implementations[index]

### 3.3 Modify implementation

#### 3.3.1 Modify implementation metadata 

In [46]:
implementation.notes = 'These files were provided by Henan Zhou (Mississippi State) on July 13, 2021 and posted with her permission.'

In [61]:
implementation.key

'fcfb8310-51bd-48dd-81ac-ae54d4b7c319'

In [ ]:
#display(HTML(implementation.html()))

In [55]:
implementation.notes += '  Update May 26 2021: This version is not compatible for LAMMPS versions starting with 29 Oct 2020 due to Infinify and NaN values no longer allowed.'

In [56]:
implementation.status='superseded'

In [ ]:
#implementation.type='LAMMPS pair_style sw',
#implementation.key=None,
#implementation.id='2009--Molinero-V--water--ipr-1',
#implementation.status=None,
#implementation.date=datetime.date(2020, 1, 10), 
implementation.notes= ' '.join(['The table files were sent by Xianbao Duan (Huazhong Univ. of Sci. and Tech) on 18 June 2020 and posted with his permission. The example.lammps.in file gives an example of the LAMMPS pair_style and pair_coeff lines that can be used. A copy of Ta_Zhou04.eam.alloy from 2004--Zhou-X-W--Ta--LAMMPS--ipr2 is included here for completeness.',
                                ])

In [ ]:
implementation.date=datetime.date(2014, 11, 24)

In [ ]:
implementation.notes = 'This file was provided by Sergei Starikov (Ruhr-Universität Bochum, Germany) on 5 May 2019. It has been carefully tested and gives the expected property predictions. Update March 15, 2020: This version was identified to not be compatible with LAMMPS versions after 7 Aug 2019 due to more rigorous format checks.'

In [ ]:
implementation.notes += ' Update Jan 15, 2020: It was noticed that the original file hosted here was truncated and incomplete.'
implementation.notes += ' The incomplete file will not work with LAMMPS versions after 7 Aug 2019.'
implementation.notes += ' For earlier LAMMPS versions, both versions of the parameter file appear to behave identically.'

In [ ]:
implementation.status = 'retracted'
implementation.notes = 'This file was sent by J. Kullgren (Uppsala University) on 19 December 2016 and posted with his permission. Update March 15, 2020: This version was identified to not be compatible with LAMMPS.'

In [ ]:
implementation.notes='These files were posted on March 15, 2020 by Lucas Hale. They modify the above version by separating the comments into a separate file, making the parameter file compatible with LAMMPS.'

In [ ]:
implementation.type = 'LAMMPS pair_style reax/c'

In [ ]:
implementation.id = '2013--Bonny-G--Fe-Cr-W--LAMMPS--ipr1'

#### 3.3.2 Add implementation content

The content for the implementation can exist in one of the following formats

1. Artifacts: files that are hosted on the potentials repository.
2. Parameters: list of the potential's parameters to show as html text on the repository.
3. WebLinks: hyperlinks to content hosted externally.

#### Option #1: Add artifacts

File path parameters

- __newpotpath__ is the path to the directory where the new potential files are located.
- __localpath__ is the path to the local copy of the NIST potentials repository.
- __webpath__ is the url path for the NIST potentials repository that coincides with the localpath on the machine.
- __relpath__ is the relative path from localpath where the files are copied to, and the relative path from webpath where they will be found once uploaded to the website.

In [25]:
# Paths to files on desktop
machine = socket.gethostname()
if machine == 'PG903193':
    newpotpath = Path('E:/website/new potentials')
    localpath = Path('E:/website/IPR-website/potentials/')

# Paths to files on laptop
elif machine == 'PG902164':
    newpotpath = Path('C:/Users/lmh1/Documents/website/new potentials')
    localpath = Path('C:/Users/lmh1/Documents/website/IPR-website/potentials/')

# Web and relative paths
webpath = 'https://www.ctcms.nist.gov/potentials/'
relpath = f'Download/{potential.id}/{index+1}/'

Each artifact has:

- __filename__ (*str*) The name of the file (no path information).
- __url__ (*str*) The absolute url where the file can be downloaded from.
- __label__ (*str, optional*) An optional descriptive label for the file.

In [26]:
# Specify files from newpotpath to add
filenames = [
    Path(newpotpath, 'Fe_Earth_core.eam.fs'),
    #Path(newpotpath, 'Bi.library.meam'),
    ]

labels = [None for i in range(len(filenames))]

In [38]:
# Specify files from newpotpath to add
filenames = [
    Path(newpotpath, 'READ_ME.txt'),
    Path(newpotpath, 'FeCrW_d.eam.alloy'),
    Path(newpotpath, 'FeCrW_s.eam.fs'),
    ]

labels = [
    'Documentation',
    'd_band',
    's_band']

In [27]:
# Make download directory in local copy of website
downloadpath = Path(localpath, relpath)
if not downloadpath.is_dir():
    downloadpath.mkdir(parents=True)

# Copy files and add artifact listings
for filename, label in zip(filenames, labels):
    # Build url
    url = webpath + relpath + filename.name
    # Copy files
    shutil.copy(filename, downloadpath)
    print(Path(downloadpath, filename.name))
    
    # Add artifact listing to implementation
    implementation.add_artifact(filename=filename.name, url=url, label=label)

E:\website\IPR-website\potentials\Download\2021--Mendelev-M-I--Fe\1\Fe_Earth_core.eam.fs


#### Option #2: Add parameters

Each parameter has:

- __name__ (*str, optional*) The name of the parameter or string parameter line.
- __value__ (*float, optional*) The value of the parameter.
- __unit__ (*str, optional*) Units associated with value.

Note: if the parameters are purely text-based, you can use only the name field.

#### Option #3: Add weblinks

Each weblink has:

- __url__ (*str*) URL for the link.
- __label__ (*str, optional*) A short description label to proceed the link.
- __linktext__ (*str, optional*) The text for the link, i.e. what gets clicked on.

## 4. Review and save potentials

In [50]:
potential.html(render=True)

In [ ]:
potential.build_model()

In [51]:
db.save_potential(potential, overwrite=True, verbose=True)
db.upload_potential(potential, workspace=workspace, overwrite=True, verbose=True)
complete_potentials.append(potential)

Potential record named potential.2021--Mendelev-M-I--Fe updated in C:\Users\lmh1\Documents\library
Potential record named potential.2021--Mendelev-M-I--Fe updated in https://potentials.nist.gov/
Potential record named potential.2021--Mendelev-M-I--Fe assigned to workspace Global Public Workspace


In [ ]:
#db.delete_potential(potential, local=True, remote=True, verbose=True)

In [ ]:
db.download_potentials(overwrite=True, format='json', indent=4, verbose=True)

In [24]:
complete_potentials.append(potential)

## 5. potential_LAMMPS

This section generates the potential_LAMMPS metadata records for generating the LAMMPS input command lines for supported potentials.

### 5.1 Build potential_LAMMPS

#### Specify parameters

Common settings

- __id__ (*str, optional*) implementation id. Should match with the implementation id from above.
- __key__ (*str, optional*) implementation key. Should match with the implementation key from above.
- __potid__ (*str, optional*) potential id. Should match with the potential id from above.
- __potkey__ (*str, optional*) potential key. Should match with the potential key from above.
- __elements__ (*list, optional*) The elements that the implementation simulates.  Should match with the elements from above if the implementation explicitly models each element.
- __symbols__ (*list, optional*) The particle model symbols defined by the implementation.  Not needed if identical to elements.
- __masses__ (*list, optional*) The masses to assign to each symbol/element.  Should be given if the implementation specifies the mass.  Required if elements is not given.
- __units__ (*str, optional*) The LAMMPS units to use with the implementation.  Default value is 'metal'.
- __atom_style__ (*str, optional*) The LAMMPS atom_style to use with the implementation.  Default value is 'atomic'.
- __pair_style__ (*str*) The LAMMPS pair_style associated with the implementation.
- __pair_style_terms__ (*list, optional*) Any additional terms that should appear on the pair_style command line.
- __comments__ (*str, optional*) Info about where the potential is hosted, to be printed when LAMMPS commands are generated.
- __dois__ (*list, optional*) A list of publication DOIs associated with the potential. 
- __artifacts__ (*list, optional*) A list of Artifact objects for the associated potential files.

In [31]:
kwargs = {}
kwargs['id'] = implementation.id
kwargs['key'] = implementation.key
kwargs['potid'] = potential.id
kwargs['potkey'] = potential.key

kwargs['elements'] = ['Fe']#potential.elements
#kwargs['symbols'] = ['Fe', 'nmFe']
kwargs['masses'] = [55.845]

#kwargs['units'] =
#kwargs['atom_style'] = 

kwargs['pair_style'] = implementation.type.replace('LAMMPS pair_style ', '')
#kwargs['pair_style_terms'] = ['polar_off']

kwargs['comments'] = '\n'.join([f'Potential {implementation.id} listed in the NIST Interatomic Potentials Repository:',
                                f'https://www.ctcms.nist.gov/potentials/entry/{potential.id}/{implementation.id}.html',
                                f''])

#kwargs['dois'] = []
#for cit in potential.citations:
#    kwargs['dois'].append(cit.bib['doi'])

kwargs['artifacts'] = implementation.artifacts

Option #1 For pair potentials without parameter files, e.g lj/cut

- __interactions__ (*list*) Lists parameters associated with each unique interaction.  Each value is a dict with
    - __symbols__ (*list*) The two elemental model symbols to associate with the interaction.
    - __terms__ (*list*) The terms that appear on the pair_coeff command line for that interaction.

In [ ]:
kwargs['interactions'] = [
    {'symbols': ['Al', 'Al'], 'terms': [1.23, 3.412]},
    {'symbols': ['Al', 'Cu'], 'terms': [1.23, 3.412]},
    {'symbols': ['Cu', 'Cu'], 'terms': [1.23, 3.412]},
]

Option #2 For potentials with single parameter files, e.g. eam/alloy

- __paramfile__ (*str*) The name (no path) of the potential's parameter file.

In [32]:
for file in filenames:
    print(file.name)

Fe_Earth_core.eam.fs


In [33]:
kwargs['paramfile'] = 'Fe_Earth_core.eam.fs'

Option #3 For potentials with a parameter file and a library file, e.g. meam

- __libfile__ (*str*) The name (no path) of the potential's library parameter file.
- __paramfile__ (*str, optional*) The name (no path) of the potential's parameter file, if one is used.

In [27]:
kwargs['libfile'] = 'Bi.library.meam'
kwargs['paramfile'] = 'Bi.parameter.meam'

Option #4 For classic EAM potentials, i.e. eam

- __paramfiles__ (*list*) The names (no paths) of the parameter files for each unique element. Order should match order of elements.

In [ ]:
kwargs['paramfiles'] = []

Option #5 For OpenKIM potentials

- __kimid__ (*str*) The KIM model ID.

In [ ]:
kwargs['kimid'] = 

#### Build potential_LAMMPS

In [34]:
lammps_potential = potentials.build_lammps_potential(**kwargs).potential()

### 5.2 Load potential_LAMMPS from database 

In [17]:
lammps_potential = db.get_lammps_potential(id='2013--Bonny-G--Fe-Cr-W--LAMMPS--ipr2', verbose=True)

Found 1 matching potential_LAMMPS records in local library
No KIM potentials added: list of models is empty
Matching record retrieved from local


### 5.3 Load potential_LAMMPS from local file not in database 

In [ ]:
fname = Path('C:/Users/lmh1/Documents/library/potential_LAMMPS/2019--Fischer-F--Cu-Ni--LAMMPS--ipr2.json')

lammps_potential = potentials.PotentialLAMMPS(fname)

### 5.4 Review and save

#### Review model

In [11]:
print(lammps_potential.model.json(indent=2))

{
  "potential-LAMMPS": {
    "key": "8ffc1d8f-8b64-4d14-8803-86a83121628c",
    "id": "2011--Bonny-G--Fe-Ni-Cr--LAMMPS--ipr1",
    "status": "superseded",
    "potential": {
      "key": "17676f3f-5287-4bb9-892a-0bfe9b1fb160",
      "id": "2011--Bonny-G-Terentyev-D-Pasianot-R-C-et-al--Fe-Ni-Cr",
      "doi": "10.1088/0965-0393/19/8/085008"
    },
    "comments": "Potential 2011--Bonny-G--Fe-Ni-Cr--LAMMPS--ipr1 listed in the NIST Interatomic Potentials Repository:\nhttps://www.ctcms.nist.gov/potentials/entry/2011--Bonny-G-Terentyev-D-Pasianot-R-C-et-al--Fe-Ni-Cr/2011--Bonny-G--Fe-Ni-Cr--LAMMPS--ipr1.html\n",
    "units": "metal",
    "atom_style": "atomic",
    "atom": [
      {
        "element": "Fe",
        "mass": 55.845
      },
      {
        "element": "Ni",
        "mass": 58.6934
      },
      {
        "element": "Cr",
        "mass": 51.9961
      }
    ],
    "pair_style": {
      "type": "eam/alloy"
    },
    "pair_coeff": {
      "term": [
        {
          "file": 

In [36]:
print(lammps_potential.pair_info())

print "Potential 2021--Mendelev-M-I--Fe--LAMMPS--ipr1 listed in the NIST Interatomic Potentials Repository:"
print "https://www.ctcms.nist.gov/potentials/entry/2021--Mendelev-M-I--Fe/2021--Mendelev-M-I--Fe--LAMMPS--ipr1.html"
print "Parameter file(s) can be downloaded at:"
print "https://www.ctcms.nist.gov/potentials/Download/2021--Mendelev-M-I--Fe/1/Fe_Earth_core.eam.fs"
pair_style eam/fs
pair_coeff * * Fe_Earth_core.eam.fs Fe
mass 1 55.845




#### Save

In [19]:
db.save_lammps_potential(lammps_potential, filenames=filenames,  verbose=True)
db.upload_lammps_potential(lammps_potential, workspace=workspace, overwrite=True, verbose=True)

potential_LAMMPS record named 2013--Bonny-G--Fe-Cr-W--LAMMPS--ipr2 updated in https://potentials.nist.gov/
potential_LAMMPS record named 2013--Bonny-G--Fe-Cr-W--LAMMPS--ipr2 assigned to workspace Global Public Workspace


In [ ]:
#db.delete_lammps_potential(lammps_potential, local=True, remote=True, verbose=True)

In [ ]:
db.download_lammps_potentials(getfiles=True, overwrite=True, verbose=True)

- - -

## 5. Actions

Actions are used by the website to note changes in content and representation. 

### 5.1 Build action

#### Option #1: new posting(s)

In [38]:
for complete_potential in complete_potentials:
    print(complete_potential.id)

2021--Mendelev-M-I--Fe


In [39]:
action = potentials.load_record('Action',
    type = 'new posting',
    potentials = complete_potentials[:1],
   # date = None,
    comment = "New posting for Fe in the Earth's core",
)

#### Option #2: updated posting(s)

In [69]:
action = potentials.load_record('Action',
    type = 'updated posting',
    potentials = complete_potentials,
    date = None,
    comment = 'LAMMPS versions updated to replace INF and NaN values with real numbers',
)

#### Option #3: retraction

In [ ]:
action = potentials.load_record('Action',
    type = 'retraction',
    potentials = complete_potentials,
    date = None,        
    comment =,
)

#### Option #4: site change

In [ ]:
action = potentials.load_record('Action',
    type = 'site change',
    date = None,
    comment = 'IDs added for all non-LAMMPS implementations. The handling of fictional potentials made consistent with "real" potentials.',
    potentials = []
)

### 5.2 Look and save

In [40]:
action.html(render=True)

In [ ]:
action.build_model()

In [41]:
db.upload_action(action, workspace=workspace, overwrite=True, verbose=True)

Action record named 2021-07-16 New posting for Fe in the Earth's core added to https://potentials.nist.gov/
Action record named 2021-07-16 New posting for Fe in the Earth's core assigned to workspace Global Public Workspace


- - -

## 6. Requests

Requests are used by the website to list when people are looking for specific models that we don't have.

### 6.1 Build request

In [ ]:
request = potentials.Request(
    #comment = 'reactive potentials',
    date = datetime.date(2020, 1, 2),
    systems = [
        {
            #'formula':,
            'elements': ['Mg', 'Cu'],
        },
        #{
        #    'formula': 'Moose',
       #     'elements': ['Ti', 'Cl'],
       # },
    ]
)

### 6.2 Look and save

In [ ]:
display(HTML(request.html()))

In [ ]:
model = request.build_model()
elements = ' '.join(model.finds('element'))

title = f'{request.date} {elements}'
content = model.xml()
template = 'Request'

db.cdcs.upload_record(content=content, template=template, title=title, workspace=workspace)

## 7. Related models

In [ ]:
import json

def add_related_models(localpath, interaction, model1, model2):
    
    with open(Path(localpath, 'site/related-interactions.json')) as f:
        related_models = json.load(f)
    
    # If no shared models for that interaction yet
    if interaction not in related_models:
        print('new interaction')
        related_models[interaction] = [[model1, model2]]
            
    else:
        int_models = related_models[interaction]
        
        # Search for set with one of the elements
        match = False
        for i in range(len(int_models)):
            modelset = int_models[i]
            if model1 in modelset:
                if match is False:
                    match = i
                else:
                    raise ValueError('given models found in multiple sets!')
            if model2 in modelset:
                if match is False or match == i:
                    match = i
                else:
                    raise ValueError('given models found in multiple sets!')
                    
        if match is False:
            print('new set')
            int_models.append(sorted([model1, model2]))
        else:
            if model1 not in int_models[match]:
                int_models[match] = sorted(int_models[match] + [model1])
                print('model1 added to existing set')
            elif model2 not in int_models[match]:
                int_models[match] = sorted(int_models[match] + [model2])
                print('model2 added to existing set')
            else:
                print('both models already in existing set')
    
    with open(Path(localpath, 'site/related-interactions.json'), 'w') as f:
        json.dump(related_models, fp=f)

In [ ]:
interaction = 'Ni'
model1 = '2004--Mishin-Y--Ni-Al'
model2 = '2019--Fischer-F-Schmitz-G-Eich-S-M--Cu-Ni'
machine = 'desktop'

if machine == 'desktop':
    localpath = Path('E:/website/IPR-website/potentials/')
elif machine == 'laptop':
    localpath = Path('C:/Users/lmh1/Documents/website/IPR-website/potentials/')

add_related_models(localpath, interaction, model1, model2)

In [3]:
l = [ 4, 4, 6]

In [4]:
None in l

False